# Inter-document similarity measures


## 1. Word Embeddings 
### We want to project words (word/token vectors) onto numeric vectors such that we can apply mathematical algorithms and methods on them

#### 1.1 Prerequisites

In [5]:
import scipy
import numpy as np
import gensim
import time
import matplotlib.pyplot as plt

from ast import literal_eval
from preprocess_data import processed_data
from numpy.linalg import norm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics.cluster import normalized_mutual_info_score
from gensim import corpora
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors
import gensim.downloader as api

#### 1.2 BoW, Tf-Idf, Word2Vec using Skipgram and CBOW. 

In [104]:
#convert token list to bow-vector from scratch
def text2bow2(tk1,tk2):
    dictionary = corpora.Dictionary()
    vec = []
    vec1 = []
    vec2 = []
    vocab = []
    for t1,t2 in zip(tk1,tk2):
        vocab.append(t1)
        vocab.append(t2)    
    for v in vocab:
        vec1.append(tk1.count(v))
        vec2.append(tk2.count(v))
    vec.append(vec1)
    vec.append(vec2)
    return vec

def identity_tokenizer(text):
    return text

#convert token list to tf-idf-vector using sklearn
def text2tfidf(token_lists):
    tfidf_vectorizer = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False)
    tfidf_vecs = tfidf_vectorizer.fit_transform(token_lists)
    return tfidf_vecs.todense()

#convert token list to word2vec using gensim
#training algorithm is defined in w2v, sg=0 meaning skip-gram, 1 meaning CBOW
def text2w2v_sg(tk1,tk2):
    token_list=[]
    for t1,t2 in zip(tk1,tk2):
        token_list.append(t1)
        token_list.append(t2)
    model = Word2Vec(token_list, window=5, min_count=1, workers=4, sg=1)
    #model.train(token_lists,total_examples=len(token_lists),epochs=10)
    return model

def text2w2v_cb(tk1,tk2):
    token_list=[]
    for t1,t2 in zip(tk1,tk2):
        token_list.append(t1)
        token_list.append(t2)
    model = Word2Vec(token_list, window=5, min_count=1, workers=4, sg=1)
    return model

def loadGlv():
    glove2word2vec(glove_input_file="./data/glove.6B.300d.txt", word2vec_output_file="gensim_glove_vectors.txt")
    glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)
    return glove_model


## 2. Cosine Similarity

### We want to compute the cosine similarity between documents as a measure for inter-document similarity.
### We will compute the cossim first using the BoW embedding, word2vec with SkipGram and CBoW, tf-idf and finally with the pre-trained GloVe 

In [110]:
#we set the similarity threshold to 0.8 to avoid false positives
def cos_similarities_bow(processed_data):
    similarities = []
    #compute pairwise similarities between bow vectors
    for (str1,str2) in zip(processed_data[1:,3],processed_data[1:,4]):
        bows = text2bow2(str1,str2)
        cosine = 1 - scipy.spatial.distance.cosine(np.array(bows[0]), np.array(bows[1]))
        if cosine >= 0.8:
            similarities.append('1')
        else:
            similarities.append('0')
    return np.array(similarities)

#threshold here is higher, as numeric vectors seem to be very close to each other -> not suitable!
def cos_similarities_w2v(processed_data, algorithm_str):
    similarities = []
    if algorithm_str=="skipgram":
        model= text2w2v_sg(processed_data[1:,3],processed_data[1:,4])
    else:
        model= text2w2v_cb(processed_data[1:,3],processed_data[1:,4])
    #compute pairwise similarities between skipgram vectors
    for (str1,str2) in zip(processed_data[1:,3],processed_data[1:,4]):
        vec1 = np.mean([model[word] for word in str1 if word in model],axis=0)
        vec2 = np.mean([model[word] for word in str2 if word in model],axis=0)
        cosine = scipy.spatial.distance.cosine(vec1, vec2)
        if 1-cosine > 0.994:
            similarities.append('1')
        else:
            similarities.append('0')
    return np.array(similarities)


def cos_similarities_tfidf(processed_data):
    similarities = []
    #compute pairwise similarities between tf-idf vectors
    for (str1,str2) in zip(processed_data[1:,3],processed_data[1:,4]):
        pair = []
        pair.append(str1)
        pair.append(str2)
        tfidf_matrix = text2tfidf(pair)
        similarity = tfidf_matrix * tfidf_matrix.T
        if similarity[0,1] >= 0.8:
            similarities.append('1')
        else:
            similarities.append('0')
    return np.array(similarities)

#using the pre-trained vectors from Wikipedia database from GloVe
#train the gloVe model with the vectors and applying model on each token of a sentence 
#to generate numeric vectors
def cos_similarities_glove(processed_data):
    similarities = []
    glove_model = loadGlv()
    for (str1,str2) in zip(processed_data[1:,3],processed_data[1:,4]):
        vec1 = np.mean([glove_model[word] for word in str1 if word in glove_model],axis=0)
        vec2 = np.mean([glove_model[word] for word in str2 if word in glove_model],axis=0)
        cosine = scipy.spatial.distance.cosine(vec1, vec2)
        if 1-cosine >= 0.8:
            similarities.append('1')
        else:
            similarities.append('0')
    return np.array(similarities)

#calculating normalized mutual info score from sklearn to measure the accuracy of the results
print("NMI score, Cossim, BoW embedding:",normalized_mutual_info_score(cos_similarities_bow(processed_data), processed_data[1:,0]))   
print("NMI score, Cossim, W2V with SkipGram:",normalized_mutual_info_score(cos_similarities_w2v(processed_data,"skipgram"), processed_data[1:,0])) 
print("NMI score, Cossim, W2V with CBoW:",normalized_mutual_info_score(cos_similarities_w2v(processed_data,"cbow"), processed_data[1:,0])) 
print("NMI score, Cossim, Tf-Idf embedding:",normalized_mutual_info_score(cos_similarities_tfidf(processed_data), processed_data[1:,0])) 
print("NMI score, Cossim, pre-trained Glove embedding:",normalized_mutual_info_score(cos_similarities_glove(processed_data), processed_data[1:,0])) 

NMI score, Cossim, BoW embedding: 0.06392774474703444


<ipython-input-110-4452d86a4750>:23: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  vec1 = np.mean([model[word] for word in str1 if word in model],axis=0)
<ipython-input-110-4452d86a4750>:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vec1 = np.mean([model[word] for word in str1 if word in model],axis=0)
<ipython-input-110-4452d86a4750>:24: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  vec2 = np.mean([model[word] for word in str2 if word in model],axis=0)
<ipython-input-110-4452d86a4750>:24: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vec2 = np.mean([model[word] for word in str2 if word in model],axis=0)


NMI score, Cossim, W2V with SkipGram: 0.012826149818436385


<ipython-input-110-4452d86a4750>:23: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  vec1 = np.mean([model[word] for word in str1 if word in model],axis=0)
<ipython-input-110-4452d86a4750>:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vec1 = np.mean([model[word] for word in str1 if word in model],axis=0)
<ipython-input-110-4452d86a4750>:24: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  vec2 = np.mean([model[word] for word in str2 if word in model],axis=0)
<ipython-input-110-4452d86a4750>:24: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vec2 = np.mean([model[word] for word in str2 if word in model],axis=0)


NMI score, Cossim, W2V with CBoW: 0.014549726680304647
NMI score, Cossim, Tf-Idf embedding: 0.0129055029389371
NMI score, Cossim, pre-trained Glove embedding: 0.04162622040210457


From the results following ranking can be made:
1. BoW
2. GloVe 
3. CBoW 
4. Tf-Idf 
5. SkipGram

# 3. Latent Semantic Indexing
## Reducing dimensionality of document vectors

In [32]:
def cos_similarities_LSI(processed_data):
    similarities = []
    tokens = []
    for (i,j) in zip(processed_data[1:,3],processed_data[1:,4]):
        tokens.append(i)
        tokens.append(j)
    common_dictionary = corpora.Dictionary(tokens)       
    common_corpus = [common_dictionary.doc2bow(tk) for tk in tokens] 
    model = LsiModel(common_corpus, id2word=common_dictionary)
    doc_lsi = model[common_corpus]
    for (str1,str2) in zip(processed_data[1:,3],processed_data[1:,4]):
        str1 = common_dictionary.doc2bow(str1)
        str2 = common_dictionary.doc2bow(str2)
        vec1 = np.asarray(model[str1])
        vec2 = np.asarray(model[str2])
        cosine = scipy.spatial.distance.cosine(vec1[:,1], vec2[:,1])
        if 1-cosine >= 0.85:
            similarities.append('1')
        else:
            similarities.append('0')
    return np.array(similarities)

print("NMI score, LSI + Cossim:",normalized_mutual_info_score(cos_similarities_LSI(processed_data), processed_data[1:,0])) 

NMI score, LSI + Cossim: 0.023942868585615545


Apparently LSI also yields bad results.

Thoughts.. maybe I need to add dim reduction methods? 

## 4. Smooth Inverse Frequency

## 5. Word Mover´s Distance

In [2]:
model = api.load('word2vec-google-news-300')


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [14]:
def compute_wmd(processed_data):
    labels = []
    for (str1,str2) in zip(processed_data[1:,3],processed_data[1:,4]):
        distance = model.wmdistance(str1, str2)
        if distance < 1.25:
            labels.append(1)
        else:
            labels.append(0)
    return np.array(labels)

print("NMI score, WMD from gensim:",normalized_mutual_info_score(compute_wmd(processed_data), processed_data[1:,0])) 

NMI score, WMD from gensim: 0.056496349119139616


## 8. BERT algorithm

In [2]:
import numpy as np
from sent2vec.vectorizer import Vectorizer

train_data = open('./data/msr_paraphrase_train.txt', 'r', encoding='latin-1')
unprocessed_train_data = np.array([example.split("\t") for example in train_data.readlines()])


In [8]:
import random 
batch_size = 200
batch = np.array(random.sample(list(unprocessed_train_data), batch_size))
#print(batch[0:,4])

from scipy import spatial

labels = []

for (sentence1,sentence2) in zip(batch[190:,3],batch[190:,4]):
    sentences = []
    sentences.append(sentence1)
    sentences.append(sentence2)
    vectorizer = Vectorizer()
    vectorizer.bert(sentences)
    vectors = vectorizer.vectors
    cos = dist = spatial.distance.cosine(vectors[0], vectors[1])
    if cos < 0.5:
        labels.append(1)
    else:
        labels.append(0)

print(labels)

#print(pair_lefts)
#print(pair_rights)

#vectorizer.bert(pair_rights)

#vectors_rights = vectorizer_rs.vectors


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [26]:
print(np.array(labels)[:2])
print(np.array(batch[190:192,0]))
print("NMI score, WMD from gensim:",normalized_mutual_info_score(np.array(labels), np.array(batch[190:,0]))) 

[1 1]
['1' '1']
NMI score, WMD from gensim: 1.6639952752053812e-15


In [27]:
pair_lefts = []
pair_rights = []

from scipy import spatial

labels = []

for (sentence1,sentence2) in zip(unprocessed_train_data[1:,3],unprocessed_train_data[1:,4]):
    sentences = []
    sentences.append(sentence1)
    sentences.append(sentence2)
    vectorizer = Vectorizer()
    vectorizer.bert(sentences)
    vectors = vectorizer.vectors
    cos = dist = spatial.distance.cosine(vectors[0], vectors[1])
    if cos < 1:
        labels.append(1)
    else:
        labels.append(0)

print(labels)

#print(pair_lefts)
#print(pair_rights)

#vectorizer.bert(pair_rights)

#vectors_rights = vectorizer_rs.vectors

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 